In [11]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from newspaper import Article
import re


#ticker_list = pd.read_csv('ticker_list.txt',header=None)
ticker_list = ['RIO', 'XOM', 'GE', 'F', 'MO', 'XRX', 'GS', 'HBC', 'JPM', 'LYG', 'MS', 'RF']
headers = {'User-Agent':
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) '
               'AppleWebKit/537.36 (KHTML, like Gecko)'
               'Chrome/39.0.2171.95 '
               'Safari/537.36'}

for ticker in ticker_list:
    linklist = []
    page = 1
    
    # crawl all transcripts links for a single ticker 
    while True:
        url = 'https://seekingalpha.com/symbol/'+ ticker + '/earnings/more_transcripts?page=' + str(page)
        wb = requests.get(url, headers=headers)
        soup = BeautifulSoup(wb.text, 'lxml')
        if not soup.select('div div a'):
            break
        else:
            for i in soup.select('div div a'):
                linklist.append('https://seekingalpha.com' + i.get('href').strip('\"\\') + '?part=single')
        
        page += 1
        time.sleep(10)
    time.sleep(30)
    
    print('All links got\n')

    
    df = pd.DataFrame(columns=['Ticker', 'Date', 'Time'])
    df_link = pd.DataFrame(columns=['Link', 'Fail_Link'])
    
    for ind, link in enumerate(linklist):
        print(link)
        art = Article(link, language='en', fetch_images=False, memoize_articles=False, browser_user_agent='Mozilla/5.0')
        time.sleep(5)
        art.download()
        
        # test whether downloading succeed and store links to link dataframe
        try:
            art.parse()
            df_link.loc[ind, 'Link'] = link
        except:
            print(link)
            df_link.loc[ind, 'Fail_Link'] = link
            continue
        content = re.split(r'\n+', art.text)
        
        try:
            file_name = ticker + content[1] + '.txt'
            df.loc[i, 'Ticker'] = ticker
            df.loc[i, 'Date'] = content[1]
            df.loc[i, 'Time'] = content[2]
            
            with open('/Users/Maxwell/PycharmProjects/Github/ProjectBigData/output/'+file_name, 'w') as f:
                f.write(content)
            
        except IndexError:
            print(content)
            print(i)
            continue 
        
        if not df.empty:
            df.to_csv('time_data.csv', mode='a')
        
        if not df_link.empty:
            df.to_csv('link_data.csv', mode='a')
        time.sleep(10)

All links got

https://seekingalpha.com/article/4058308-rio-tinto-plc-2016-q4-results-earnings-call-slides?part=single


['with this article? Please tell us . Disagree with this article? Submit your own']
<a href='\"/article/6807-rio-tinto-q4-2005-earnings-conference-call-transcript-rtp\"' sasource='\"qp_analysis\"' target='\"_blank\"'>Rio Tinto Q4 2005 Earnings Conference Call Transcript (RTP)</a>
https://seekingalpha.com/article/4043797-rio-tintos-rio-ceo-jean-sebastien-jacques-q4-2016-results-earnings-call-transcript?part=single


TypeError: write() argument must be str, not list

['https://seekingalpha.com/article/4056229-exxon-mobils-xom-ceo-darren-woods-hosts-2017-analyst-meeting-transcript?part=single',
 'https://seekingalpha.com/article/4041222-exxon-mobils-xom-q4-2016-results-earnings-call-transcript?part=single',
 'https://seekingalpha.com/article/4040958-exxon-mobil-corporation-2016-q4-results-earnings-call-slides?part=single',
 'https://seekingalpha.com/article/4016510-exxon-mobils-xom-q3-2016-results-earnings-call-transcript?part=single',
 'https://seekingalpha.com/article/4016329-exxon-mobil-corporation-2016-q3-results-earnings-call-slides?part=single',
 'https://seekingalpha.com/article/3993426-exxon-mobil-corporation-2016-q2-results-earnings-call-slides?part=single',
 'https://seekingalpha.com/article/3993326-exxon-mobil-xom-q2-2016-results-earnings-call-transcript?part=single',
 'https://seekingalpha.com/article/3969448-exxon-mobil-xom-q1-2016-results-earnings-call-transcript?part=single',
 'https://seekingalpha.com/article/3857646-exxon-mobil-xom-